In [1]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func


In [ ]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/top_ten_historic_5y.csv')
fundamental_df_clean = pd.read_csv('Resources/02_ASX_Fundamental_Final_Clean.csv')
industry_groups_df.head()

In [ ]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
print(top_ten_historic_df['Ticker'].isnull().sum())

In [ ]:
industry_groups_df.info()

In [ ]:
fundamental_df_clean.info()

In [ ]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [ ]:
top_ten_historic_df.info()

In [ ]:
#top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date']).apply(lambda x: x.date())

In [ ]:
#print(top_ten_historic_df['Date'].unique())

In [ ]:
industry_groups_df.info()

In [ ]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)
    
class Fundamental(Base):
    __tablename__ = 'fundamental'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String,ForeignKey('top_ten.ticker'))
    lastPrice = Column(Float)         
    Change = Column(String)       
    Bid_Ask = Column(String)           
    volumePerDay = Column(Float)         
    volume4wAvg = Column(Float)          
    Open = Column(Float)              
    dayRange = Column(String)              
    prevClose = Column(String)        
    lastTrade = Column(String)            
    oneWeek = Column(String)             
    oneMonth = Column(String)           
    YTD2023 = Column(String)            
    oneYear = Column(String)            
    vsSectorOneYr = Column(String)         
    vsASX200OneYr = Column(String)         
    marketCap = Column(Float)          
    ASXRank = Column(String)            
    sectorRank = Column(String)         
    sharesIssued = Column(Float)       
    Sector = Column(String)               
    similarCompanies = Column(String)     
    EPS = Column(Float)                 
    DPS = Column(Float)                
    bookValuePerShare = Column(Float)    
    Breakdown = Column(String)          
    Recommendation = Column(String)       
    lastUpdated = Column(String)                     
    PE= Column(Float) 
        
#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)



In [2]:
engine = create_engine('sqlite:///Resources/top_ten_asx.db')

In [ ]:
from sqlalchemy.orm import Session

# Insert DataFrames into Database
#Session = sessionmaker(bind=engine)
session = Session(bind=engine)

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))

# Insert ASX_Fundamental
session.bulk_insert_mappings(Fundamental, fundamental_df_clean.to_dict(orient="records"))




In [ ]:
# #commit changes
#session.commit()

In [ ]:
# #Query the Database and Create DataFrame
# # Example: Querying the TopTen table and creating a DataFrame
# top_ten_query = session.query(TopTen).all()
# asx = pd.DataFrame([{
#     'ticker': record.ticker,
#     'company_name': record.company_name,
#     'market_cap': record.market_cap,
#     'industry_id': record.industry_id
# } for record in top_ten_query])

# print(asx)


In [ ]:
#import dependencies for inspector
from sqlalchemy import inspect

In [ ]:
# Instantiate an Inspector with the engine
inspector = inspect(engine)

In [ ]:
# Get a list of table names
table_names = inspector.get_table_names()
print(table_names)

In [ ]:
#Get columns of top_ten_historic
columns = inspector.get_columns('top_ten')
for column in columns:
    print(column['name'])

In [3]:
#reflect database into ORM classes 
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['fundamental', 'top_ten', 'industry_groups', 'top_ten_historic']

In [ ]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

In [ ]:
#create a database session object
session = Session(bind =engine)

In [ ]:
#list all of the tickers found in the top_ten table 
session.query(top_ten.ticker).all()